# Check incidence and elevation angle calculations

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import nisarHDF
import matplotlib.pyplot as plt
import numpy as np
import geojson
import os
from scipy import optimize
import pyproj
from datetime import datetime
import h5py
import scipy

In [1]:
!wget 'https://nisar.asf.alaska.edu/NISAR-SAMPLE-DATA/RUNW/ALOS-1_Rosamond_20081012_20081127/NISAR_L1_PR_RUNW_001_005_A_219_220_4020_SH_20081012T060910_20081012T060926_20081127T060959_20081127T061015_P01101_M_F_J_001.h5'

--2024-02-10 17:48:14--  https://nisar.asf.alaska.edu/NISAR-SAMPLE-DATA/RUNW/ALOS-1_Rosamond_20081012_20081127/NISAR_L1_PR_RUNW_001_005_A_219_220_4020_SH_20081012T060910_20081012T060926_20081127T060959_20081127T061015_P01101_M_F_J_001.h5
Resolving nisar.asf.alaska.edu (nisar.asf.alaska.edu)... 108.138.94.36, 108.138.94.4, 108.138.94.107, ...
Connecting to nisar.asf.alaska.edu (nisar.asf.alaska.edu)|108.138.94.36|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://d1mv8zhcvry6x4.cloudfront.net/s3-7fdfbcb0ce308dc58d08f97acb1f0590/sds-n-cumulus-prod-nisar-sample-data.s3.us-west-2.amazonaws.com/RUNW/ALOS-1_Rosamond_20081012_20081127/NISAR_L1_PR_RUNW_001_005_A_219_220_4020_SH_20081012T060910_20081012T060926_20081127T060959_20081127T061015_P01101_M_F_J_001.h5?A-userid=None&Expires=1707619694&Signature=snNdZfSEsk0khLqMszsrLo2yIXtgCTiKZYeum~O4Zqd1rFQusP9QFXZQyvRiZEEud45BtfQiLVpD4wv~AV5WdSpIs-Y4JOFyHBS2LcrpVsiordd9AaCCbJbifPjiSPcbH~GZOAvXyyBZe4eTmTXdjCd3Qr

In [2]:
#hdfFile = 'icesheet/data_pair2_ALOS2497735530-230811_ALOS2499805530-230825_14d/RUNW_0000526644_001001_ALOS2497735530-230811_0000528214_001001_ALOS2499805530-230825.h5'
hdfFile = 'NISAR_L1_PR_RUNW_001_005_A_219_220_4020_SH_20081012T060910_20081012T060926_20081127T060959_20081127T061015_P01101_M_F_J_001.h5'
h5 = h5py.File(hdfFile, 'r')
geolocation = h5['science']['LSAR']['RUNW']['metadata']['geolocationGrid']

NameError: name 'h5py' is not defined

## Get Statevectors

In [151]:
def parseStateVectors(h5):
    orbit = h5['science']['LSAR']['RUNW']['metadata']['orbit']
    sv = {}
    time = np.array(orbit['time'])
    position = np.array(orbit['position'])
    velocity = np.array(orbit['velocity'])
    for i, pos, vel in zip(range(0, 3), ['xsv', 'ysv', 'zsv'], ['vxsv', 'vysv', 'vzsv']):
        sv[pos]= scipy.interpolate.RegularGridInterpolator(
                    [time], position[:, i], method='quintic')
        sv[vel]= scipy.interpolate.RegularGridInterpolator(
                    [time], velocity[:, i], method='quintic')
    return sv

In [152]:
sv = parseStateVectors(h5)

## Get Grid

In [153]:
slantRange = np.array(geolocation['slantRange'])
dR = slantRange[1]- slantRange[0]
zeroDopplerTime = np.array(geolocation['zeroDopplerTime'])
heightAboveEllipsoid = np.array(geolocation['heightAboveEllipsoid'])
#
elevationAngle =  np.array(geolocation['elevationAngle'])
incidenceAngle = np.array(geolocation['incidenceAngle'])
#
print(elevationAngle.shape, heightAboveEllipsoid.shape, zeroDopplerTime.shape, slantRange.shape)


(21, 251, 58) (21,) (251,) (58,)


Get radar geometry from tables. 

In [154]:
iz, it, ir = 2, 0, 0
zPt = heightAboveEllipsoid[iz]
slantRangePt = slantRange[ir]
zeroDopplerPt = zeroDopplerTime[it]
elevationAnglePt = elevationAngle[iz, it, ir]
incidenceAnglePt = incidenceAngle[iz, it, ir]
print(f'z = {zPt} (m), zeroDopplerTime=  {zeroDopplerPt:.5f} (s), slantRange = {slantRangePt} (m), elevationAngle {elevationAnglePt:.4f} incidenceAngle {incidenceAnglePt:.4f}')

z = 0 (m), zeroDopplerTime=  22150.33200 (s), slantRange = 743587.3917643395 (m), elevationAngle 18.9604 incidenceAngle 21.1295


Get target position.

In [155]:
xCoord = np.array(geolocation['coordinateX'])
yCoord = np.array(geolocation['coordinateY'])
XYToECEF = pyproj.Transformer.from_crs("EPSG:3031", "EPSG:4978").transform
xPt, yPt = xCoord[iz, it, ir], yCoord[iz, it, ir]

ptECEF = XYToECEF(xPt, yPt, zPt)
ptRadius = np.linalg.norm(ptECEF)
print(f'Radius to pt target at elevation {zPt} m =  {ptRadius:.2f} m')

Radius to pt target at elevation 0 m =  6363497.91 m


Get sv position

In [156]:
satPosition = [sv[coord]([zeroDopplerPt]) for coord in ['xsv', 'ysv', 'zsv']]
print(satPosition)
ReH = np.linalg.norm(satPosition)
print(f'Radius from Earth Center to Sat {ReH:.2f}')

[array([-3015234.50300603]), array([-5058048.2991014]), array([3913152.85956166])]
Radius from Earth Center to Sat 7070237.41


In [157]:
r_b_t = ptRadius
r_b_sc_plus_h = ReH
rho = slantRangePt
bta =  np.arccos((0.5*r_b_t/(r_b_sc_plus_h))+0.5*((r_b_sc_plus_h)/r_b_t)-(0.5*(rho/(r_b_sc_plus_h))*(rho/r_b_t)))
#  cthta = (r_b_t**2 - r_b_sc_plus_h**2 - rho**2)/ (-2. *  r_b_t * rho)
cthta = (rho**2 + r_b_sc_plus_h**2 - r_b_t**2 ) / (2. *  r_b_sc_plus_h * rho)
thta = np.arccos(cthta)
print(f"beta  = {bta*180./np.pi:.4f}")
print(f"theta = {thta*180./np.pi:.4f}")
print(f"incidence angle = {(thta+bta)*180./np.pi:.4f}")

beta  = 1.9748
theta = 17.1515
incidence angle = 19.1263


In [158]:
elevationAngleCalc = np.arccos((slantRangePt**2 + ReH**2 - ptRadius**2) / (2.0 * slantRangePt * ReH))
incidenceAngleCalc = np.arcsin(np.sin(elevationAngleCalc) * ReH/ptRadius)
print('Calculated 1')
print(f'elevationAngle = {np.degrees(elevationAngleCalc):.4f}, incidenceAngle={np.degrees(incidenceAngleCalc):.4f}')
print('Calculated 2')
print(f'elevationAngle = {thta*180./np.pi:.4f}, incidenceAngle={(thta+bta)*180./np.pi:.4f}')
print('From Table')
print(f'elevationAngle = {elevationAnglePt:.4f}, incidenceAngle={incidenceAnglePt:.4f}')

Calculated 1
elevationAngle = 17.1515, incidenceAngle=19.1263
Calculated 2
elevationAngle = 17.1515, incidenceAngle=19.1263
From Table
elevationAngle = 18.9604, incidenceAngle=21.1295


In [159]:
print(np.degrees(elevationAngleCalc) - elevationAnglePt)

-1.8088818739317212
